In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# API Key
api_key = "PLACE KEY HERE"

In [2]:
weather_df = pd.read_csv("../Outputs/WeatherPy.csv")
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bluff,-46.6000,168.3333,63.81,67,20,1.92,NZ,1651629522
1,jati,24.3539,68.2671,81.34,78,2,14.67,PK,1651629523
2,poum,-20.2333,164.0167,77.97,78,6,7.78,NC,1651629523
3,busselton,-33.6500,115.3333,60.48,48,5,13.94,AU,1651629402
4,bethel,41.3712,-73.4140,52.65,85,94,5.01,US,1651629524
...,...,...,...,...,...,...,...,...,...
682,warrnambool,-38.3833,142.4833,55.81,64,100,15.66,AU,1651630014
683,ovalle,-30.5983,-71.2003,67.01,51,92,0.72,CL,1651629933
684,invermere,50.5167,-116.0354,54.73,65,100,4.74,CA,1651630014
685,aksu,41.1231,80.2644,72.99,15,100,0.69,CN,1651630015


In [3]:
# Configure gmaps
gmaps.configure(api_key)

In [4]:
# Store the latitude and longitude locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity as the weight
humidity = weather_df["Humidity"]

In [5]:
# Plot the heatmap
heat_map = gmaps.figure(center = [0.0, -0.0], zoom_level = 2)

# Add a heat layer to the heatmap
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Now add that layer
heat_map.add_layer(heat_layer)

heat_map

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Narrow down the cities to fit weather conditions
max_temp = weather_df.loc[(weather_df['Max Temp']>=70) & (weather_df['Max Temp']<=80)]

wind_speed = max_temp.loc[(weather_df['Wind Speed']<=10)]

true_weather = wind_speed.loc[(wind_speed['Cloudiness']==0)]

true_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
132,taree,-31.9000,152.4667,77.11,52,0,3.04,AU,1651629614
133,buraydah,26.3260,43.9750,79.72,18,0,10.00,SA,1651629617
164,port macquarie,-31.4333,152.9167,76.89,63,0,8.72,AU,1651629643
195,benguela,-12.5763,13.4055,75.20,75,0,2.80,AO,1651629667
284,riyadh,24.6877,46.7219,77.14,19,0,8.43,SA,1651629490


In [7]:
# Store a variable called hotel_df
hotel_df = true_weather.loc[:,["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
132,taree,AU,-31.9000,152.4667,
133,buraydah,SA,26.3260,43.9750,
164,port macquarie,AU,-31.4333,152.9167,
195,benguela,AO,-12.5763,13.4055,
284,riyadh,SA,24.6877,46.7219,
304,lima,PE,-12.0432,-77.0282,
312,jiangyin,CN,31.9282,120.3126,
316,natchez,US,31.5604,-91.4032,
351,nelson bay,AU,-32.7167,152.1500,
359,cookeville,US,36.1628,-85.5016,


In [8]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : api_key}

print("-------Begin Search-------")

for index, row in hotel_df.iterrows():
        
    lat = row["Lat"]
    
    lng = row["Lng"]
    
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel for {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # Show if there is no hotel close by
    except (KeyError, IndexError):
        print("Oops, try again")
    
print("-------End of Search-------")

-------Begin Search-------
Closest hotel for taree is Manning River Hotel.
Closest hotel for buraydah is Best Western Plus Buraidah.
Closest hotel for port macquarie is Macquarie Waters Boutique Apartment Hotel.
Closest hotel for benguela is Hotel Misinga.
Closest hotel for riyadh is Radisson Blu Hotel, Riyadh.
Closest hotel for lima is Sheraton Lima Hotel & Convention Center.
Closest hotel for jiangyin is Jiangyin International Hotel.
Closest hotel for natchez is Magnolia Bluffs Casino Hotel, BW Premier Collection.
Closest hotel for nelson bay is Seabreeze Hotel.
Closest hotel for cookeville is Holiday Inn Express & Suites Cookeville, an IHG Hotel.
Oops, try again
Closest hotel for healdsburg is Hotel Healdsburg.
Closest hotel for pasni is 3G hotel pasni.
Closest hotel for caravelas is Pousada Liberdade.
Closest hotel for huainan is Gujing International Hotel.
Oops, try again
Closest hotel for huntsville is AC Hotel by Marriott Huntsville Downtown.
-------End of Search-------


In [24]:
# Show the results
hotel_df.dropna(subset=['Hotel Name'])

hotel_df

,City,Country,Lat,Lng,Hotel Name
132,taree,AU,-31.9000,152.4667,Manning River Hotel
133,buraydah,SA,26.3260,43.9750,Best Western Plus Buraidah
164,port macquarie,AU,-31.4333,152.9167,Macquarie Waters Boutique Apartment Hotel
195,benguela,AO,-12.5763,13.4055,Hotel Misinga
284,riyadh,SA,24.6877,46.7219,"Radisson Blu Hotel, Riyadh"
304,lima,PE,-12.0432,-77.0282,Sheraton Lima Hotel & Convention Center
312,jiangyin,CN,31.9282,120.3126,Jiangyin International Hotel
316,natchez,US,31.5604,-91.4032,"Magnolia Bluffs Casino Hotel, BW Premier Colle..."
351,nelson bay,AU,-32.7167,152.1500,Seabreeze Hotel
359,cookeville,US,36.1628,-85.5016,"Holiday Inn Express & Suites Cookeville, an IH..."


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
new_marker = gmaps.marker_layer(locations, info_box_content=[f"Hotel location: {rate}" for rate in hotel_info])

heat_map.add_layer(new_marker)

# Save the figure
plt.savefig("Heat Map.png")

# Display figure
heat_map

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>